# Simple trading strategy using Simple Moving Average

## Install packages

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install pandas

In [ ]:
%pip install matplotlib

In [ ]:
%pip install scipy

## Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Loading data

In [ ]:
fb = pd.read_csv('./data/facebook.csv', index_col='Date')
ms = pd.read_csv('./data/microsoft.csv', index_col='Date')

## Create PriceDiff column

Price diff between two days

In [ ]:
fb['PriceDiff'] = fb['Close'].shift(-1) - fb['Close']
fb['PriceDiff']

In [ ]:
ms['PriceDiff'] = ms['Close'].shift(-1) - ms['Close']
ms.loc['2015-01-05', 'PriceDiff']

## Create Direction column

Is price increase or decrease?

In [ ]:
fb['Direction'] = [1 if fb['PriceDiff'].loc[ei] > 0 else 0 for ei in fb.index ]
ms['Direction'] = [1 if ms['PriceDiff'].loc[ei] > 0 else 0 for ei in ms.index ]
ms.loc['2015-01-05', 'Direction']

## Draw Moving Average chart

In [ ]:
fb['ma50'] = fb['Close'].rolling(50).mean()

#plot the moving average
plt.figure(figsize=(10, 8))
fb['ma50'].loc['2015-01-01':'2015-12-31'].plot(label='MA50')
fb['Close'].loc['2015-01-01':'2015-12-31'].plot(label='Close')
plt.legend()
plt.show()

In [ ]:
ms['ma50'] = ms['Close'].rolling(50).mean()

#plot the moving average
plt.figure(figsize=(10, 8))
ms['ma50'].loc['2015-01-01':'2015-12-31'].plot(label='MA50')
ms['Close'].loc['2015-01-01':'2015-12-31'].plot(label='Close')
plt.legend()
plt.show()

## Strategy

In [ ]:
# add `MA10` and `MA50` column
fb['MA10'] = fb['Close'].rolling(10).mean()
fb['MA50'] = fb['Close'].rolling(50).mean()
fb = fb.dropna()

# add `Shares` column
fb['Shares'] = [1 if fb.loc[ei, 'MA10']>fb.loc[ei, 'MA50'] else 0 for ei in fb.index]

# add close price of tomorrow NextClose
fb['NextClose'] = fb['Close'].shift(-1)
fb['Profit'] = [fb.loc[ei, 'NextClose'] - fb.loc[ei, 'Close'] if fb.loc[ei, 'Shares']==1 else 0 for ei in fb.index]
fb['Profit'].plot()
plt.xticks(rotation = 45)
plt.axhline(y=0, color='red')

## Display wealth

Evaluate our strategy performance by looking at the total wealth we have.

In [ ]:
fb['wealth'] = fb['Profit'].cumsum()
fb['wealth'].plot()
plt.xticks(rotation = 45)
plt.title(f'Total money you win is {fb.loc[fb.index[-2], "wealth"]}')